In [1]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from keras.utils import get_file
import numpy as np
import argparse
import cv2
import os
import cvlib as cv

Using TensorFlow backend.


In [2]:
# download pre-trained model file (one-time download)
dwnld_link = "https://github.com/chillum1718/cvlib/releases/download/v0.2/gender_detection.model"
model_path = get_file("gender_detection.model", dwnld_link,
                     cache_subdir="pre-trained", cache_dir=os.getcwd())

In [3]:
model = load_model(model_path)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [4]:
webcam = cv2.VideoCapture(0)
if not webcam.isOpened():
    print("could not open webcam")
    exit()
    
classes = ['man','woman']    
    

In [5]:
# loop through frames
while webcam.isOpened():

    # read frame from webcam 
    status, frame = webcam.read()

    if not status:
        print("Could not read frame")
        exit()

    # apply face detection
    face, confidence = cv.detect_face(frame)

    print(face)
    print(confidence)

    # loop through detected faces
    for idx, f in enumerate(face):

        # get corner points of face rectangle        
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]

        # draw rectangle over face
        cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)

        # crop the detected face region
        face_crop = np.copy(frame[startY:endY,startX:endX])

        if (face_crop.shape[0]) < 10 or (face_crop.shape[1]) < 10:
            continue

        # preprocessing for gender detection model
        face_crop = cv2.resize(face_crop, (96,96))
        face_crop = face_crop.astype("float") / 255.0
        face_crop = img_to_array(face_crop)
        face_crop = np.expand_dims(face_crop, axis=0)

        # apply gender detection on face
        conf = model.predict(face_crop)[0]
        print(conf)
        print(classes)

        # get label with max accuracy
        idx = np.argmax(conf)
        label = classes[idx]

        label = "{}: {:.2f}%".format(label, conf[idx] * 100)

        Y = startY - 20 if startY - 20 > 20 else startY + 20

        # write label and confidence above face rectangle
        cv2.putText(frame, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 255, 0), 2)

    # display output
    cv2.imshow("gender detection", frame)

    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release resources
webcam.release()
cv2.destroyAllWindows()

[[313, 227, 487, 454]]
[0.98269385]
[0.99829656 0.00147878]
['man', 'woman']
[[305, 230, 484, 440]]
[0.93930626]
[0.99857295 0.00195208]
['man', 'woman']
[]
[]
[[300, 235, 480, 442]]
[0.8187796]
[0.9759178  0.03768932]
['man', 'woman']
[[296, 239, 475, 439]]
[0.7147796]
[0.9813162  0.02938651]
['man', 'woman']
[[298, 240, 473, 442]]
[0.63233536]
[0.8900832  0.16280031]
['man', 'woman']
[]
[]
[[295, 235, 471, 442]]
[0.804597]
[0.97606    0.03698799]
['man', 'woman']
[]
[]
[[296, 236, 471, 445]]
[0.64052904]
[0.9661362  0.05234385]
['man', 'woman']
[[295, 238, 470, 443]]
[0.58649796]
[0.9630034  0.05654877]
['man', 'woman']
[[297, 239, 473, 446]]
[0.8542128]
[0.97882473 0.03210227]
['man', 'woman']
[[295, 239, 473, 442]]
[0.73670137]
[0.9715887  0.04457559]
['man', 'woman']
[[294, 237, 473, 441]]
[0.6502032]
[0.97892064 0.03276788]
['man', 'woman']
[[293, 239, 473, 440]]
[0.72717386]
[0.98301333 0.02669073]
['man', 'woman']
[[293, 237, 473, 441]]
[0.7471346]
[0.97808826 0.03406625]
['man

KeyboardInterrupt: 